## Efisiensi vs Flow Coeff & Work Coeff


In [3]:
# IMPORTS
import numpy as np
import CoolProp
from CoolProp.CoolProp import PropsSI as Props
from CoolProp.CoolProp import PhaseSI as Phase
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

In [5]:
#INPUT TURBINE INLET & OUTLET FLUID STATE
def whichcycle (k):
    global T_1,T_5,P_1,P_5,mflow,fluid #Total Temperature (K),Total Pressure(P),'Nama fluida'
    if k == 1:
        T_1 = 360   
        T_5 = 335.4     #This value is obtained from Refprop. Will be replaced by value from CoolProp
        P_1 = 0.9   *1e6
        P_5 = 0.4   *1e6
        fluid = 'R245fa'
        mflow = 0.5
    if k == 2:
        T_1 = 400
        T_5 = 343.55
        P_1 = 1.22
        P_5 = 0.15079
        fluid = 'ISOPENTANE' 
        mflow = 0.5
    if k == 3:
        T_1 = 0
        T_5 = 0
        P_1 = 0
        P_5 = 0
        fluid = 0 
        mflow = 0.5
    if k == 4:
        T_1 = 0
        T_5 = 0
        P_1 = 0
        P_5 = 0
        fluid = 0 
        mflow = 0.5

In [6]:
def rcw (Alpha,Beta):   #input in radians
    return (np.cos(Alpha)/np.cos(Beta))
def ruc (Alpha,Beta):
    return (1/(np.sin(Alpha)+np.tan(Beta)*np.cos(Alpha)))
def cu (ruc,rcw):
    return (1-(ruc*rcw)**2+ruc**2)

In [93]:
def Compute(flow_coeff,work_coeff,k):
    global  T_1,T_5,P_1,P_5
    global  NR,r4,Alpha4,b4,Ct4,rho4,Beta5,beta5didconverge,Beta4opt
    global cu5,cu5num,cu4,errorbeta5,beta,U5,U4,r5,DeltaH,C4,C5,W4,W5,C5_0
    global Cm4,Ct4,U4,Alpha4,Beta4,Cm5,Ct5,U5,Alpha5,Beta5,beta5didconverge
    global rho04s,rho4s,a4,C4,h5ss,h05ss,h04s,h4s,a01,C5didconverge,kC5,errorC5
    global TotalLoss,LossInc,LossPass,LossTip,LossWind,LossTE,LossExit
    rath5   = 0.4
    rats5   = 0.75
    rpm     = 20000
    Zratio  = 0.35
    NR      = 8
    whichcycle (k)       # The cycle to be computed is the k
    Cp4 = Props('C','T',T_1,'P',P_1,fluid)
    Cv4 = Props('O','T',T_1,'P',P_1,fluid)
    gamma = Cp4/Cv4
    Rx = 8.31446261815324   #J/K.mol

    #General Properties inlet outlet turbin
    H_1     = Props('H','T',T_1,'P',P_1,fluid)     #J/kg
    s1     = Props('S','T',T_1,'P',P_1,fluid)     #J/kg.K 
    T_5     = Props('T','P',P_5,'S',s1,fluid)  # =>asumsi nozzle isenthalpy DAN Isentropic
    H_5     = Props('H','T',T_5,'P',P_5,fluid)  # meski pada kenyataannya isenthalpic nozzle tidak isentropic
    DeltaH  = H_1-H_5            #Ideal === Isentropic Total Enthalpy change 
    #rho04s    = Props('D','T',T_1,'P',P_1,fluid)  #rho4  ~= rho1?
    #rho05s    = Props('D','T',T_5,'P',P_5,fluid)  #kg/m3
    #rho dihitung pada static
    C0s     = np.sqrt(2*DeltaH)         #Spouting Velocity

    #Segitiga Kecepatan Inlet, m/s, radians
    Beta4   = 0                         # =>INPUT Control Variable(rads)
    U4      = np.sqrt(DeltaH/work_coeff)
    Cm4     = U4*flow_coeff
    W4      = Cm4/np.cos(Beta4)
    Ct4     = U4-W4*np.sin(Beta4) 
    Alpha4  = np.arctan(Ct4/Cm4)  #in radians           
    C4      = Cm4/np.cos(Alpha4)

    #Perhitungan Properties ideal lain (Total)
    p01     = P_1           #inlet volute [1], Total
    T01     = T_1
    h01     = H_1
    h02s    = H_1           #inlet nozzle [2], Total
    s02s    = s1            #ideal volute === approx. as isentropic
    p02s    = p01
    T02s    = T01
    h03s    = h02s           #outlet nozzle [3], Total
    s03s    = s02s            #ideal nozzle === approx. as isentropic (in Total)
    p03s    = p02s
    T03s    = T02s
    h04s    = h03s           #inlet rotor [4], Total
    s04s    = s03s            #outlet nozzle === inlet rotor
    p04s    = p03s
    T04s    = T03s

    h05ss   = H_5
    p05ss   = P_5
    T05ss   = T_5
    s05ss   = Props('S','H',h05ss,'P',p05ss,fluid)

    #Perhitungan Properties ideal lain (Static)
    h4s     = h04s-1/2*C4**2
    p4s     = Props('P','H',h4s,'S',s04s,fluid)
    T4s     = Props('T','H',h4s,'S',s04s,fluid)
    rho4s   = 2*(p04s-p4s)/C4**2
    a01     = Props('A','P',p01,'T',T01,fluid)
    a4      = Props('A','P',p4s,'T',T4s,fluid)
    


    #Perhitungan Geometri 
    r4  = U4/np.radians(rpm*6)
    D4  = r4*2
    Zr  = Zratio*r4
    rh5 = rath5*r4      #to be changed soon,
    rs5 = rats5*r4      #to be changed soon, ga pake rats
    r5  = (rs5+rh5)/2
    b5 = rs5-rh5
    b4 = mflow/(2*np.pi*r4*rho4s*C4)

    #Segitiga Kecepatan Outlet
    # Alpha5  = np.radians(1)    # =>INPUT initial Control Variable(rads)
    #U5      = r5*np.radians(rpm*6)
    #cu4     = cu(ruc(Alpha4,Beta4),rcw(Alpha4,Beta4))
    #cu5     = (U4**2*cu4-2*DeltaH)/U5**2
    #     #Mencari nilai Beta5 dengan galat cu < 0.5%
    # beta5didconverge = False    #initial state
    # Beta5 = np.degrees(0.1)
    # cu5num = cu(ruc(Alpha5,Beta5),rcw(Alpha5,Beta5))
    # errorbeta5  = np.abs((cu5-cu5num)/cu5)
    # while errorbeta5>0.005:
    #     Beta5 = Beta5+np.radians(0.01)
    #     cu5num = cu(ruc(Alpha5,Beta5),rcw(Alpha5,Beta5))
    #     errorbeta5  = np.abs((cu5-cu5num)/cu5)
    #     if errorbeta5<=0.01:
    #         beta5didconverge = True
    #     if Beta5 >= np.radians(180):
    #         break
    #==> dicoba dulu pakai uct
    #Ct5     = (U4*Ct4-DeltaH)/U5
    #Cm5 = U4/(np.tan(Alpha5)+np.tan(Beta5))
    #W5  = Cm5/np.cos(Beta5)
    #C5  = Cm5/np.cos(Alpha5)
    #Ct5 = Cm5*np.tan(Alpha5)
    # Re4     = rho4s*C4*b4/Props('V','P',p04s,'T',T04s,fluid)
    # #Properties ideal static di 5
    # h5ss    = h05ss-1/2*C5**2
    # p5ss    = Props('P','H',h5ss,'S',s05ss,fluid)
    # rho5ss  = 2*(p05ss-p5ss)/C5**2

    Re4     = rho4s*C4*b4/Props('V','P',p04s,'T',T04s,fluid)
    #Segitiga Kecepatan outlet versi 2
    Alpha5  = np.radians(3)     #
    U5      = r5*np.radians(rpm*6)
    C5didconverge = False
    C5_0    = (U4*Ct4-DeltaH)/(np.sin(Alpha5)*U5) #i=0
    C5ii    = C5_0  #trick untuk menghemat baris
    #errorC5 = np.abs((C5i-C5ii)/C5i)
    kC5     = 0
    while C5didconverge == False:
        kC5     = kC5 +1
        C5i     = C5ii
        h5ss    = h05ss-1/2*C5**2
        p5ss    = Props('P','H',h5ss,'S',s05ss,fluid)
        rho5ss  = 2*(p05ss-p5ss)/C5**2
        C5ii    = mflow/(2*np.pi*b5*r5*rho5ss)
        errorC5 = np.abs((C5i-C5ii)/C5i)
        if errorC5 <= 0.005:
            C5didconverge = True
            C5  = C5ii
    Cm5     = C5*np.cos(Alpha5)
    Ct5     = C5*np.sin(Alpha5)
    Beta5   = np.arctan((U5-Ct5)/Cm5)
    W5      = Cm5/np.cos(Beta5)




    # \\\\\\\ <<---------<<----||----->>------------>> ////////
    ## Losses Coefficient ##

    #Rotor Incidence Losses
    Beta4opt = np.arctan((-1.98/NR)/NR/(1-1.98/NR)*np.tan(Alpha4))
    LossInc = 0.5*(W4**2)*(np.sin(np.abs(Beta4-Beta4opt)))**2  #m2/s2

    #Rotor Passage Losses
    LH = np.pi/4*((Zr-b4/2)+(r4-rh5-b5/2))                                                              #m
    DH = 0.5*((4*np.pi*r4*b4/(2*np.pi*r4+Zr*rh5))+((2*np.pi*(rs5**2-rh5**2)/(np.pi*(rs5-rh5))+Zr*b5)))  #m
    Y5 = np.arctan(0.5*(np.tan(Beta4)+np.tan(Beta5)))
    C = Zr/np.cos(Y5)
    if (r4-rs5)/b5>=0.2:
        KpCETI = 0.11
    else:
        KpCETI = 0.22
    LossPass = KpCETI*(LH/DH+0.68*((1-(r5/r4)**2)*np.cos(Beta5)/(b5/C))*((W4**2+W5**2)/2))
    
    #Rotor Clearance Losses
    Ca = (1-(rs5/r4))/(Cm4*b4)
    Cr = (rs5/r4)*((Zr-b4)/(Cm5*r5*b5))
    Ka = 0.4
    Kr = 0.75
    Kar = -0.3
    Ea = 0.0003
    Er = 0.0003
    LossTip = (U4**3*NR/(8*np.pi))*(Ka*Ea*Ca+Kr*Er*Cr+Kar*np.sqrt(Ea*Er*Ca*Cr))

    #Windage Losses
    Eb = 0.0003
    Kf = 3.7*(Eb/r4)**0.1/Re4**0.5
    LossWind = Kf*((rho4s+rho5ss)/2)*U4**3*r4**2/(2*mflow*W5**2)

    #Trailing Edge Losses
    tb4 = 0.04*r4
    tb5 = 0.02*r4
    LossTE = rho5ss*W5**2/2*(NR*tb5/(np.pi*(rh5+rs5)*np.cos(Beta5)))**2
    
    #Exit Losses
    LossExit = 0.5*C5**2    # => mungkin untuk ubah dari total jadi static. abaikan dulu

    # => Sum Enthalpy Losses
    TotalLoss = LossInc + LossPass + LossTip + LossWind + LossTE
    # \\\\\\\ <<---------<<----||----->>------------>> ////////


    #Perhitungan Properties considering losses
    h05 = h05ss#TotalLoss#bentar masih salah lossnya       #nozzle masih diasumsikan isentropic dan isenthalpic
    h5  = h05-1/2*C5**2     
    p05 = p05ss
    T05 = Props('T','H',h05,'P',p05,fluid)
    p5  = p5ss
    T5  = Props('T','H',h5,'P',p5,fluid)



    #Real states calculatins
    p05 = P_5       #Ganti symbol
    T05 = Props('T','H',h05,'P',p05,fluid)
    s5  = Props('S','P',p05,'T',T05,fluid)
    T5  = Props('T','H',h5,'S',s5,fluid)
    p5  = Props('P','S',s5,'H',h5,fluid)







In [83]:

print(cu(ruc(0,np.radians(70)),rcw(0,np.radians(70))))
Compute(0.2,1.1,2)
print("DeltaH=",DeltaH)
print((Ct4*U4-Ct5*U5))
print(np.abs(DeltaH-((Ct4*U4-Ct5*U5)))/DeltaH*100,"%")
print(r4)
print(r5)
print(U4)
print(U5)
print(cu4)
print(fluid)
print("cu5=",cu5)
print("cu5num=",cu5num)
print("converge?=",beta5didconverge)
print("Error Beta5=",errorbeta5)
print("Beta5=",np.degrees(Beta5))

print(TotalLoss)
print(LossInc,LossPass,LossTip,LossWind,LossTE)

3.0531133177191805e-16


UnboundLocalError: local variable 'C5i' referenced before assignment

In [94]:
Compute(0.1,2,1)
print(h04s)
print(h4s)
print(h05ss)
print(h5ss)
print("w(by enthalpy)=",DeltaH)
print("w(by enthalpy)=",h04s-h05ss)
print("w(by vel)=",-1/2*((U5**2-U4**2)+(C5**2-C4**2)-(W5**2-W4**2)))
print("W(by velCtU)=",Ct4*U4-Ct5*U5)
print("W(by velCtU)=",Ct4*U4-C5_0*np.sin(Alpha5)*U5)
print(C4/a01)
print(C4/a4)
print("Alpha4=",np.degrees(Alpha4))
print("Beta4=",np.degrees(Beta4))
print("Cm4=",Cm4)
print("C4=",C4)
print("U4=",U4)

print("Alpha5=",np.degrees(Alpha5))
#print("beta5converge?=",beta5didconverge)
print("C5 converged? =",C5didconverge)
print("iterated for",kC5,"times")
print("error C5=",errorC5)
print("Beta5=",np.degrees(Beta5))
print("Cm5=",Cm5)
print("C5=",C5)
print("U5=",U5)
print("Beta4opt=",np.degrees(Beta4opt))

468829.27699595
465029.7811156753
453781.7685592186
453723.1580622576
w(by enthalpy)= 15047.508436731354
w(by enthalpy)= 15047.508436731354
w(by vel)= 7495.493187451264
W(by velCtU)= 7495.493187451257
W(by velCtU)= 15047.508436731354
0.6768425591878666
0.6616149120175562
Alpha4= 84.28940686250037
Beta4= 0.0
Cm4= 8.673957700130705
C4= 87.1721960291775
U4= 86.73957700130705
Alpha5= 3.0000000000000004
C5 converged? = True
iterated for 2 times
error C5= 0.0
Beta5= 77.63234425474616
Cm5= 10.812026417001361
C5= 10.826864258184148
U5= 49.87525677575156
Beta4opt= -22.349012958598337


In [ ]:
C